In [2]:
#uselful snorkel sources:
#https://www.snorkel.org/use-cases/01-spam-tutorial#2-writing-labeling-functions-lfs 3a) keyword lookup
#https://www.snorkel.org/use-cases/01-spam-tutorial#2-writing-labeling-functions-lfs 3e) preprocessor


## Setup

# Imports

In [3]:
!pip install openpyxl

In [4]:
!pip install ipywidgets

^C
ERROR: Operation cancelled by user


In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import pickle
from tqdm import tqdm

from functools import lru_cache

from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

import warnings

## Datasets

In [2]:
from load_datasets import load_asset_ds
from load_datasets import load_automets_ds
from load_datasets import load_benchls_ds
from load_datasets import load_britannica_ds
from load_datasets import load_dwikipedia_ds
from load_datasets import load_ewsewgmpm_ds
from load_datasets import load_ewsewturk_ds
from load_datasets import load_htss_ds
from load_datasets import load_hutssf_ds
from load_datasets import load_massalign_ds
from load_datasets import load_metaeval_ds
from load_datasets import load_mturksf_ds
from load_datasets import load_nnseval_ds
from load_datasets import load_onestopenglish_ds
from load_datasets import load_pwkp_ds
from load_datasets import load_semeval07_ds
from load_datasets import load_simpa_ds
from load_datasets import load_simpeval_ds
from load_datasets import load_sscorpus_ds
from load_datasets import load_turkcorpus_ds
from load_datasets import load_wikiauto_ds
from load_datasets import load_wikimanual_ds
from load_datasets import load_wikisplit_ds
from load_datasets import load_wikipediav1_ds
from load_datasets import load_wikipediav2_ds
from load_datasets import path_to_datasets

if not os.path.isdir(path_to_datasets):
    os.mkdir(path_to_datasets)

asset = load_asset_ds()
automets = load_automets_ds()
benchls = load_benchls_ds()
britannica = load_britannica_ds()
dwikipedia = load_dwikipedia_ds()
ewsewgmpm = load_ewsewgmpm_ds()
ewsewturk = load_ewsewturk_ds()
htss = load_htss_ds()
hutssf = load_hutssf_ds()
massalign = load_massalign_ds()
metaeval = load_metaeval_ds()
mturksf = load_mturksf_ds()
nnseval = load_nnseval_ds()
onestopenglish = load_onestopenglish_ds()
pwkp = load_pwkp_ds()
semeval07 = load_semeval07_ds()
simpa = load_simpa_ds()
simpeval = load_simpeval_ds()
sscorpus = load_sscorpus_ds()
turkcorpus = load_turkcorpus_ds()
wikiauto = load_wikiauto_ds()
wikimanual = load_wikimanual_ds()
wikisplit = load_wikisplit_ds()
wikipediav1 = load_wikipediav1_ds()
wikipediav2 = load_wikipediav2_ds()

combined_dataset = pd.concat([asset, automets, benchls, britannica, dwikipedia, ewsewgmpm, ewsewturk, htss, hutssf, massalign, 
                              metaeval, mturksf, nnseval, onestopenglish, pwkp, semeval07, simpa, simpeval, sscorpus, turkcorpus, 
                              wikiauto, wikimanual, wikisplit, wikipediav1, wikipediav2], axis=0).reset_index()
print(combined_dataset)

with open('/' + path_to_datasets + '/combined_dataset.pkl', 'wb') as f:
    pickle.dump(combined_dataset, f)

/usr/local/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


sl.BnGgW0UxMV4RIzFlzjZXsHUhO35w1Zk1W3q2PAJ9Oz3cXIxk_W96ZbjBmZL7b6rcd3M0KjEXN2JgnhWD6-Jl31pjZruFydtF-ZLPzi2RzRk-CRtEc9AtAYKITyBA0_3-0LXmAe5TPauxsB0

None


## Meaning Preservation

In [ ]:
from usb_utils import remove_stop_words
from usb_utils import check_sent_sim
from usb_utils import core_preserved_meaning_max_depth_5


In [ ]:
# check impact of stopwords on similarity
print(check_sent_sim(str(htss_dataset.iloc[3].orig_snt),str(htss_dataset.iloc[3].simp)))
print(check_sent_sim(remove_stop_words(htss_dataset.iloc[3].orig_snt),remove_stop_words(htss_dataset.iloc[3].simp)))

In [ ]:
# test with simplified paragraph
print(check_sent_sim(str(htss_dataset.iloc[3].orig_snt),str(htss_dataset.iloc[3].simp)))
print(core_preserved_meaning_max_depth_5(str(htss_dataset.iloc[3].orig_snt),str(htss_dataset.iloc[3].simp)))

In [ ]:
# test with simplified paragraph
print(check_sent_sim(str(htss_dataset.iloc[3].orig_snt),str(htss_dataset.iloc[3].simp)))
print(core_preserved_meaning_max_depth_5(remove_stop_words(htss_dataset.iloc[3].orig_snt),remove_stop_words(htss_dataset.iloc[3].simp)))

### comparison of effectiveness and efficiency: max depth, stopwords

In [ ]:
tqdm.pandas()
pwkp_dataset_meaning_test = pwkp_dataset.sample(100)

In [ ]:
# no max depth
pwkp_dataset_meaning_test["meaning_pres"] = pwkp_dataset_meaning_test.progress_apply(lambda row: check_sent_sim(row["orig_snt"], row["simp"]), axis=1)
pwkp_dataset_meaning_test["core_meaning_pres"] = pwkp_dataset_meaning_test.progress_apply(lambda row: core_preserved_meaning(row["orig_snt"], row["simp"])[0], axis=1)
print(pwkp_dataset_meaning_test.core_meaning_pres.describe())
print(pwkp_dataset_meaning_test.meaning_pres.describe())

In [ ]:
# max depth 5
pwkp_dataset_meaning_test2 = pwkp_dataset_meaning_test
pwkp_dataset_meaning_test2["meaning_pres"] = pwkp_dataset_meaning_test.progress_apply(lambda row: check_sent_sim(row["orig_snt"], row["simp"]), axis=1)
pwkp_dataset_meaning_test2["core_meaning_pres"] = pwkp_dataset_meaning_test.progress_apply(lambda row: core_preserved_meaning_max_depth_5(row["orig_snt"], row["simp"])[0], axis=1)
print(pwkp_dataset_meaning_test2.core_meaning_pres.describe())
print(pwkp_dataset_meaning_test2.meaning_pres.describe())

In [ ]:
# no max depth, stopwords removed
pwkp_dataset_meaning_test["meaning_pres"] = pwkp_dataset_meaning_test.progress_apply(lambda row: check_sent_sim(remove_stop_words(row["orig_snt"]), remove_stop_words(row["simp"])), axis=1)
pwkp_dataset_meaning_test["core_meaning_pres"] = pwkp_dataset_meaning_test.progress_apply(lambda row: core_preserved_meaning(remove_stop_words(row["orig_snt"]), remove_stop_words(row["simp"]))[0], axis=1)
print(pwkp_dataset_meaning_test.core_meaning_pres.describe())
print(pwkp_dataset_meaning_test.meaning_pres.describe())

In [ ]:
from scipy.stats import ttest_ind
#perform independent two sample t-test
ttest_ind(pwkp_dataset_meaning_test['core_meaning_pres'], pwkp_dataset_meaning_test['meaning_pres'])

In [ ]:
# shuffled randomly
df_random = ebbe_dataset[:104]
l = df_random.simp.to_list()
np.random.shuffle(l)
df_random["simp"] = l


df_random["meaning_pres"] = df_random.progress_apply(lambda row: check_sent_sim(row["orig_snt"], row["simp"]), axis=1)
df_random["core_meaning_pres"] = df_random.progress_apply(lambda row: core_preserved_meaning_max_depth_5(row["orig_snt"], row["simp"])[0], axis=1)

# shuffled in topic
df_in_topic_random = ebbe_dataset.loc[ebbe_dataset["label"] == 'baghdad-hum' ]
l = df_in_topic_random.simp.to_list()
np.random.shuffle(l)
df_in_topic_random["simp"] = l
for city in ['bangkok-hum', 'budapest-hum', 'buenos-hum']:
  df_t2  = ebbe_dataset.loc[ebbe_dataset["label"] == city ]
  l = df_t2.simp.to_list()
  np.random.shuffle(l)
  df_t2["simp"] = l
  df_in_topic_random = df_in_topic_random.append([df_t2],ignore_index=True)


df_in_topic_random["meaning_pres"] = df_in_topic_random.progress_apply(lambda row: check_sent_sim(row["orig_snt"], row["simp"]), axis=1)
df_in_topic_random["core_meaning_pres"] = df_in_topic_random.progress_apply(lambda row: core_preserved_meaning_max_depth_5(row["orig_snt"], row["simp"])[0], axis=1)

# not shuffled
df_in_topic = ebbe_dataset.loc[ebbe_dataset["label"] == 'baghdad-hum' ]
for city in ['bangkok-hum', 'budapest-hum', 'buenos-hum']:
  df_t2  = ebbe_dataset.loc[ebbe_dataset["label"] == city ]
  df_in_topic = df_in_topic.append([df_t2],ignore_index=True)


df_in_topic["meaning_pres"] = df_in_topic.progress_apply(lambda row: check_sent_sim(row["orig_snt"], row["simp"]), axis=1)
df_in_topic["core_meaning_pres"] = df_in_topic.progress_apply(lambda row: core_preserved_meaning_max_depth_5(row["orig_snt"], row["simp"])[0], axis=1)

In [ ]:
pwkp_dataset.sample(104)

In [ ]:
# pwkp dataset
df_pwkp = pwkp_dataset.sample(104)

df_pwkp["meaning_pres"] = df_pwkp.progress_apply(lambda row: check_sent_sim(row["orig_snt"], row["simp"]), axis=1)
df_pwkp["core_meaning_pres"] = df_pwkp.progress_apply(lambda row: core_preserved_meaning_max_depth_5(row["orig_snt"], row["simp"])[0], axis=1)

In [ ]:
# simpa dataset
df_simpa = simpa_dataset_ls.sample(104)

df_simpa["meaning_pres"] = df_simpa.progress_apply(lambda row: check_sent_sim(row["orig_snt"], row["simp"]), axis=1)
df_simpa["core_meaning_pres"] = df_simpa.progress_apply(lambda row: core_preserved_meaning_max_depth_5(row["orig_snt"], row["simp"])[0], axis=1)

In [ ]:
df = pd.DataFrame({"ebbe_rnd":df_random.core_meaning_pres.to_list(), "ebbe_in_topic_rnd":df_in_topic_random.core_meaning_pres.to_list(), "ebbe_reg":df_in_topic.core_meaning_pres.to_list(),  "pwkp_reg":df_pwkp.core_meaning_pres.to_list(),  "simpa_reg":df_simpa.core_meaning_pres.to_list()})

In [ ]:
df.boxplot()

In [ ]:
sep = []
for i in range(0,25):
  sep.append(i*0.04)
df.plot.kde(ind=sep)

In [ ]:
sep = []
for i in range(0,25):
  sep.append(i*0.04)
df.plot.kde(ind=sep)

In [ ]:
print((ttest_ind(df['pwkp_regular'], df['ebbe_regular'])))
print((ttest_ind(df['ebbe_random'], df['ebbe_regular'])))
print(ttest_ind(df['ebbe_random'], df['ebbe_in_topic_random']))
print(ttest_ind(df['ebbe_in_topic_random'], df['ebbe_regular']))

## LF Assembly

In [ ]:
from labeling_functions import get_all_lfs

In [ ]:
ABSTAIN = -1
SIMPLE = 0
NOT_SIMPLE = 1
LOST_MEANING = 2

label_map = {-1: "ABSTAIN", 0: "SIMPLE", 1: "NOT_SIMPLE", 2: "LOST_MEANING"}

In [ ]:
all_lfs = get_all_lfs()

applier = PandasLFApplier(all_lfs)
with warnings.catch_warnings():
  warnings.simplefilter("ignore")
  labels = applier.apply(sub_sample)

In [ ]:
#show some stats for the results
from snorkel.labeling import LFAnalysis
LFAnalysis(L=labels, lfs=all_lfs).lf_summary()

In [ ]:
test_l = labels[:,:250]

In [ ]:
test_l.shape

In [ ]:
#apply snorkel magic and automatically combine labels
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=labels, n_epochs=500, log_freq=5, seed=42, lr=0.001)

In [ ]:
#snorkel generates noisy labels based on the aggregation of signals provided by the labeling functions
label_model_preds = label_model.predict(L=labels)
label_model_pred_probs = label_model.predict_proba(L=labels)

In [ ]:
#explain label model
weights = label_model.get_weights()

for i in range(len(all_lfs)):
  print(f"{all_lfs[i].name} : {weights[i]}")

In [ ]:
def check_instance(id):
  print(f"src_snt : {sub_sample.iloc[id]['source_snt']}")
  print(f"simp_snt : {sub_sample.iloc[id]['simplified_snt']}")
  print()
  print("Signals:")

  for i in range(len(all_lfs)):
    print(f"{all_lfs[i].name} : {label_map[labels[i][id]]}")
  print()
  print(f"complexity_score: {label_map[label_model_preds[id]]} ({label_model_pred_probs[id]})")
  print(f"gold label : {label_map[sub_sample.iloc[id]['gold_label']]}")

In [ ]:
res = label_model.score(L=labels, Y=sub_sample['gold_label'], metrics=['accuracy', 'f1'], tie_break_policy="random")

print(f"{'Label Model Accuracy:':<25} {res['accuracy'] * 100:.1f}%")
print(f"{'Label Model F1-Score:':<25} {res['f1'] * 100:.1f}%")

In [ ]:
res

In [ ]:
LFAnalysis(labels, all_lfs).lf_summary(sub_sample['gold_label'].values)

In [ ]:
for i in range(1):
  check_instance(i)
  print('-'*30)